In [4]:
## Extract my portfolio from the csv file

import pandas as pd

# Read the CSV file
df = pd.read_csv('DivTracker_Default_1753389887657.csv')

# Display the first few rows of the DataFrame
print(df.head())

    Ticker  Quantity  Cost Per Share Currency        Date  Commission  \
0     ASML       1.0         617.400      USD  2023-07-27         NaN   
1     ASML       1.0         541.000      EUR  2023-09-26        0.65   
2     ASML      -2.0         884.100      EUR  2024-02-27        2.12   
3  BATS.GB      15.0          26.365      GBP  2023-07-24        1.98   
4  BATS.GB       5.0          26.150      GBP  2023-08-01         NaN   

  Commission Currency  DRIP Confirmed  
0                 NaN             NaN  
1                 USD             NaN  
2                 EUR             NaN  
3                 GBP             NaN  
4                 NaN             NaN  


In [5]:
def calculate_current_positions(csv_file):
    """
    Calculate current stock positions from transaction data.
    
    Parameters:
    csv_file (str): Path to the CSV file containing transaction data
    
    Returns:
    pd.DataFrame: Current positions with columns:
        - Ticker: Stock symbol
        - Current_Quantity: Number of shares currently held
        - Avg_Cost_Per_Share: Weighted average cost per share for current position
        - Currency: Primary currency for the position
        - Total_Cost_Basis: Total amount invested in current position
        - Total_Commissions: Total commissions paid for current position
        - First_Purchase: Date of first purchase
        - Last_Transaction: Date of most recent transaction
    """
    import pandas as pd
    import numpy as np
    from datetime import datetime
    
    # Read the CSV file
    df = pd.read_csv(csv_file)
    
    # Convert Date column to datetime
    df['Date'] = pd.to_datetime(df['Date'])
    
    # Fill NaN commissions with 0
    df['Commission'] = df['Commission'].fillna(0)
    
    # Initialize results list
    positions = []
    
    # Group by ticker
    for ticker in df['Ticker'].unique():
        ticker_data = df[df['Ticker'] == ticker].copy()
        ticker_data = ticker_data.sort_values('Date')
        
        # Calculate running position
        running_qty = 0
        running_cost_basis = 0
        total_commissions = 0
        transactions_for_current_position = []
        
        for _, transaction in ticker_data.iterrows():
            qty = transaction['Quantity']
            price = transaction['Cost Per Share']
            commission = transaction['Commission']
            currency = transaction['Currency']
            date = transaction['Date']
            
            total_commissions += commission
            
            if qty > 0:  # Buy transaction
                # Add to position
                running_qty += qty
                running_cost_basis += (qty * price) + commission
                transactions_for_current_position.append(transaction)
                
            else:  # Sell transaction
                qty_sold = abs(qty)
                if running_qty > 0:
                    # Calculate proportion sold
                    proportion_sold = min(qty_sold / running_qty, 1.0)
                    
                    # Reduce position
                    running_qty -= qty_sold
                    running_cost_basis *= (1 - proportion_sold)
                    
                    # Remove proportional transactions
                    remaining_transactions = []
                    for txn in transactions_for_current_position:
                        txn_qty_remaining = txn['Quantity'] * (1 - proportion_sold)
                        if txn_qty_remaining > 0.01:  # Keep transactions with significant remaining quantity
                            txn_copy = txn.copy()
                            txn_copy['Quantity'] = txn_qty_remaining
                            remaining_transactions.append(txn_copy)
                    transactions_for_current_position = remaining_transactions
        
        # Only include positions with current holdings
        if running_qty > 0.01:  # Use small threshold to handle floating point precision
            # Calculate average cost per share
            avg_cost_per_share = running_cost_basis / running_qty if running_qty > 0 else 0
            
            # Get primary currency (most common currency for this ticker)
            primary_currency = ticker_data['Currency'].mode().iloc[0] if len(ticker_data['Currency'].mode()) > 0 else ticker_data['Currency'].iloc[0]
            
            # Get first purchase and last transaction dates
            first_purchase = ticker_data[ticker_data['Quantity'] > 0]['Date'].min()
            last_transaction = ticker_data['Date'].max()
            
            positions.append({
                'Ticker': ticker,
                'Current_Quantity': round(running_qty, 6),
                'Avg_Cost_Per_Share': round(avg_cost_per_share, 2),
                'Currency': primary_currency,
                'Total_Cost_Basis': round(running_cost_basis, 2),
                'Total_Commissions': round(total_commissions, 2),
                'First_Purchase': first_purchase,
                'Last_Transaction': last_transaction
            })
    
    # Convert to DataFrame and sort by ticker
    result_df = pd.DataFrame(positions)
    if not result_df.empty:
        result_df = result_df.sort_values('Ticker').reset_index(drop=True)
    
    return result_df

# Calculate current positions
current_positions = calculate_current_positions('DivTracker_Default_1753389887657.csv')

# Display results
print("Current Portfolio Positions:")
print("=" * 80)
print(current_positions.to_string(index=False))

# Summary statistics
if not current_positions.empty:
    print(f"\nSummary:")
    print(f"Total number of positions: {len(current_positions)}")
    print(f"\nPositions by Currency:")
    currency_summary = current_positions.groupby('Currency').agg({
        'Current_Quantity': 'count',
        'Total_Cost_Basis': 'sum'
    }).rename(columns={'Current_Quantity': 'Number_of_Positions'})
    print(currency_summary)


Current Portfolio Positions:
 Ticker  Current_Quantity  Avg_Cost_Per_Share Currency  Total_Cost_Basis  Total_Commissions First_Purchase Last_Transaction
BATS.GB             170.0               24.22      GBP           4117.76              23.89     2023-07-24       2024-05-30
     BP              55.0               32.20      USD           1770.94               4.44     2024-09-03       2024-09-27
 DGE.GB             200.0               24.53      GBP           4906.19              38.64     2023-10-03       2025-07-11
 DIA.ES          135000.0                0.01      EUR           1769.74              14.74     2024-01-05       2024-09-26
    DLR              20.0              137.41      USD           2748.15               8.35     2023-04-10       2024-04-19
EBRO.ES            1150.0               16.13      EUR          18548.67              71.67     2024-04-26       2024-10-04
 ENG.ES             190.0               13.99      EUR           2658.99              16.39     2023-07

In [6]:
def simple_position_calculator(csv_file):
    """
    Simplified version that just calculates net quantities and average cost.
    """
    import pandas as pd
    
    df = pd.read_csv(csv_file)
    df['Commission'] = df['Commission'].fillna(0)
    
    # Group by ticker and calculate basic metrics
    result = df.groupby('Ticker').agg({
        'Quantity': 'sum',  # Net quantity (current holdings)
        'Cost Per Share': lambda x: (df.loc[x.index, 'Quantity'] * df.loc[x.index, 'Cost Per Share']).sum() / df.loc[x.index, 'Quantity'].sum() if df.loc[x.index, 'Quantity'].sum() != 0 else 0,  # Weighted average
        'Commission': 'sum',
        'Currency': lambda x: x.mode().iloc[0] if len(x.mode()) > 0 else x.iloc[0]
    }).round(2)
    
    # Rename columns
    result.columns = ['Net_Quantity', 'Weighted_Avg_Cost', 'Total_Commissions', 'Currency']
    
    # Filter out positions with zero or negative net quantity
    result = result[result['Net_Quantity'] > 0].reset_index()
    
    return result

# Run the simplified version
print("Simplified Position Calculation:")
print("=" * 50)
simple_positions = simple_position_calculator('DivTracker_Default_1753389887657.csv')
print(simple_positions.to_string(index=False))


Simplified Position Calculation:
 Ticker  Net_Quantity  Weighted_Avg_Cost  Total_Commissions Currency
BATS.GB         170.0              24.08              23.89      GBP
     BP          55.0              32.12               4.44      USD
 DGE.GB         200.0              24.20              38.64      GBP
 DIA.ES      135000.0               0.01              14.74      EUR
    DLR          20.0              92.81               8.35      USD
EBRO.ES        1150.0              16.07              71.67      EUR
 ENG.ES         190.0              13.91              16.39      EUR
 GAW.GB          40.0              99.40              23.66      GBP
IQQD.DE         530.0               8.68               5.52      EUR
ISPA.DE         160.0              28.90               5.55      EUR
 ITX.ES          45.0              41.42              14.67      EUR
     KO          20.0              58.94               1.41      USD
 LOG.ES          95.0              25.89              20.08      EUR
 

In [7]:
def portfolio_analysis(csv_file):
    """
    Additional portfolio analysis including closed positions and trading activity.
    """
    import pandas as pd
    
    df = pd.read_csv(csv_file)
    df['Date'] = pd.to_datetime(df['Date'])
    df['Commission'] = df['Commission'].fillna(0)
    
    print("Portfolio Analysis:")
    print("=" * 50)
    
    # All tickers ever traded
    all_tickers = df['Ticker'].unique()
    print(f"Total tickers ever traded: {len(all_tickers)}")
    
    # Current vs closed positions
    net_quantities = df.groupby('Ticker')['Quantity'].sum()
    current_positions = net_quantities[net_quantities > 0.01]
    closed_positions = net_quantities[abs(net_quantities) <= 0.01]
    
    print(f"Current positions: {len(current_positions)}")
    print(f"Closed positions: {len(closed_positions)}")
    
    # Show closed positions
    if len(closed_positions) > 0:
        print(f"\nClosed Positions (net quantity ≈ 0):")
        for ticker in closed_positions.index:
            print(f"  {ticker}: {closed_positions[ticker]:.3f}")
    
    # Trading activity by currency
    print(f"\nTrading Activity by Currency:")
    currency_activity = df.groupby('Currency').agg({
        'Quantity': 'count',
        'Commission': 'sum'
    }).round(2)
    currency_activity.columns = ['Number_of_Transactions', 'Total_Commissions']
    print(currency_activity)
    
    # Most actively traded stocks
    print(f"\nMost Actively Traded Stocks (by number of transactions):")
    activity = df.groupby('Ticker').size().sort_values(ascending=False).head(10)
    for ticker, count in activity.items():
        print(f"  {ticker}: {count} transactions")
    
    # Date range of trading activity
    print(f"\nTrading Period:")
    print(f"  First transaction: {df['Date'].min().date()}")
    print(f"  Last transaction: {df['Date'].max().date()}")
    print(f"  Trading period: {(df['Date'].max() - df['Date'].min()).days} days")

# Run portfolio analysis
portfolio_analysis('DivTracker_Default_1753389887657.csv')


Portfolio Analysis:
Total tickers ever traded: 32
Current positions: 23
Closed positions: 9

Closed Positions (net quantity ≈ 0):
  ASML: 0.000
  BKT.ES: 0.000
  MMM: 0.000
  MO: 0.000
  MSFT: 0.000
  REP.ES: 0.000
  TEF.ES: 0.000
  TROW: 0.000
  UNA.NL: 0.000

Trading Activity by Currency:
          Number_of_Transactions  Total_Commissions
Currency                                           
EUR                           82             253.01
GBP                           26              86.19
USD                           63              50.44

Most Actively Traded Stocks (by number of transactions):
  O: 21 transactions
  BATS.GB: 13 transactions
  UNA.NL: 10 transactions
  DGE.GB: 10 transactions
  MC.FR: 10 transactions
  RIO: 10 transactions
  DLR: 9 transactions
  IQQD.DE: 8 transactions
  ISPA.DE: 8 transactions
  ENG.ES: 6 transactions

Trading Period:
  First transaction: 2023-04-10
  Last transaction: 2025-07-11
  Trading period: 823 days
